In [2]:
import numpy as np 
import pandas as pd 
from sklearn.datasets import load_iris
from sklearn import preprocessing
import sklearn
from sklearn.linear_model import Lasso
from tensorflow.python.client import device_lib

import numpy as np
import pandas as pd
import sklearn
from keras.utils import to_categorical


#input csv file
training_data = pd.read_csv('training.csv', delimiter = ',') #training data frame

testing_data = pd.read_csv('scoring.csv', delimiter = ',') #testing data frame


y = np.array(training_data['rate_of_penetration'])

def parse_data(data):
    #parse out annoying strings in front 
    #training_data['segment_id'] =  training_data['segment_id'].apply(lambda x: int(x[10:]))
    data['wellbore_chev_no_id'] =  data['wellbore_chev_no_id'].apply(lambda x: int(x[17:]))
    data['area_id'] =  data['area_id'].apply(lambda x: int(x[5:]))
    data['formation_id'] =  data['formation_id'].apply(lambda x: int(x[10:]))
    data['bit_model_id'] =  data['bit_model_id'].apply(lambda x: int(x[10:]))
    return data
    

print (training_data.nunique())
print (testing_data.nunique())

training_data = parse_data(training_data.drop(['rate_of_penetration'], axis = 1))
testing_data = parse_data(testing_data)

print (training_data.shape)

training_length = training_data.shape[0] #how many training examples you have

concat = pd.concat([training_data, testing_data]) #concat the two to do one hot encoding


#one-hot encoding
def one_hot(data1):
    X = pd.get_dummies(data=data1, columns=['wellbore_chev_no_id', 'area_id', 'formation_id', 'bit_model_id', 'drillbit_size'])
    cols_to_norm = ['min_depth','max_depth', 'surface_weight_on_bit', 'surface_rpm']
    X[cols_to_norm] = X[cols_to_norm].apply(lambda x: preprocessing.scale(x)) #only standardize non-one-hot columns
    #print (X)
    X = np.array(X.drop(['segment_id','max_depth'], axis=1))
    return X

#Models

overall_one_hot = np.array(one_hot(concat)) #one-hot on the training + testing sets combined to take care of all ids

training_data = overall_one_hot[:training_length]
testing_data = overall_one_hot[training_length:]

#print (training_data)

#print (training_data.isnull().sum(axis = 0)) #no null values 



#y = preprocessing.scale(y)
X = training_data #define training matrix of examples 


from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from keras import backend
 



#grid search for parameters
from sklearn import metrics
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
%matplotlib inline
from matplotlib import pyplot as plt
from keras import optimizers #try adam and sgd
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import train_test_split
import math

    
    
def rmse(y_true, y_pred):
    return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))


#split training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0, random_state=42) #already have final test set, so train

X_test = testing_data #testing matrix of examples

model1 = Sequential()
model1.add(Dense(2, input_dim=324, activation='relu'))
model1.add(Dense(5, kernel_initializer='normal',activation='relu'))
model1.add(Dense(5, kernel_initializer='normal',activation='relu'))
model1.add(Dense(1, kernel_initializer = 'normal', activation = 'linear'))

#IMPORTANT - use this if you want to train, and comment out load weights
#sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model1.compile(loss='mse', optimizer='adam', metrics=[rmse])
#history = model1.fit(X_train, y_train, validation_split=0.05, epochs=500, batch_size=100, verbose=2)
#model1.save_weights("NN.h5")


#IMPORTANT - load pre-existing weights for best model, no need to train
model1.load_weights('NN.h5')

model2 = Sequential()
model2.add(Dense(2, input_dim=324, activation='relu'))
model2.add(Dense(7, kernel_initializer='normal',activation='relu'))
model2.add(Dense(7, kernel_initializer='normal',activation='relu'))
model2.add(Dense(1, kernel_initializer = 'normal', activation = 'linear'))
model2.compile(loss='mse', optimizer='adam', metrics=[rmse])
#history = model2.fit(X_train, y_train, validation_split=0.05, epochs=500, batch_size=150, verbose=2)
#model2.save_weights("NN2.h5")

model2.load_weights('NN2.h5')


model3 = Sequential()
model3.add(Dense(2, input_dim=324, activation='relu'))
model3.add(Dense(6, kernel_initializer='normal',activation='relu'))
model3.add(Dense(6, kernel_initializer='normal',activation='relu'))
model3.add(Dense(1, kernel_initializer = 'normal', activation = 'linear'))
model3.compile(loss='mse', optimizer='adam', metrics=[rmse])
#history = model3.fit(X_train, y_train, validation_split=0.05, epochs=500, batch_size=150, verbose=2)
#model3.save_weights("NN3.h5")

model3.load_weights('NN3.h5')


model4 = Sequential()
model4.add(Dense(2, input_dim=324, activation='relu'))
model4.add(Dense(8, kernel_initializer='normal',activation='relu'))
model4.add(Dense(8, kernel_initializer='normal',activation='relu'))
model4.add(Dense(1, kernel_initializer = 'normal', activation = 'linear'))
model4.compile(loss='mse', optimizer='adam', metrics=[rmse])
#history = model4.fit(X_train, y_train, validation_split=0.05, epochs=500, batch_size=150, verbose=2)
#model4.save_weights("NN4.h5")

#try different activation functions
#different optimizers 

model4.load_weights('NN4.h5')

import pickle

"""
params = {'n_estimators': 500, 'max_depth': 5, 'min_samples_split': 3, #3 samples fit, 250/10 vs 500/5
          'learning_rate': 0.01, 'loss': 'ls'}
clf = ensemble.GradientBoostingRegressor(**params)
clf.fit(X_train, y_train)
filename = 'GBoost1_m.sav'
pickle.dump(clf, open(filename, 'wb'))


params2 = {'n_estimators': 100, 'max_depth': 15, 'min_samples_split': 3, #3 samples fit, 250/10 vs 500/5
          'learning_rate': 0.01, 'loss': 'ls'}
clf2 = ensemble.GradientBoostingRegressor(**params2)
clf2.fit(X_train, y_train)
filename = 'GBoost2_m.sav'
pickle.dump(clf2, open(filename, 'wb'))
"""


gb1 = pickle.load(open('Gboost1_m.sav', 'rb'))
gb2 = pickle.load(open('Gboost2_m.sav', 'rb'))





y_pred5 = gb1.predict(X_test)
y_pred5 = np.reshape(y_pred5,(1567,1))
#rmse_gb = math.sqrt(mean_squared_error(y_test, y_pred5))
#print("RMSE: %.4f" % rmse_gb)


y_pred6 = gb2.predict(X_test)
y_pred6 = np.reshape(y_pred6,(1567,1))
#rmse_gb = math.sqrt(mean_squared_error(y_test, y_pred6))
#print("RMSE: %.4f" % rmse_gb)




"""
use this to calc rmse between two vectors of the same length
"""



def rmse_non_tensor(a,b):
    total = 0
    for i in range(len(a)):
        total += (a[i]-b[i])**2
    total = total/len(a)
    total = math.sqrt(total)
    return total

y_pred1 = model1.predict(X_test) #model prediction on X test set
y_pred2 = model2.predict(X_test) #model prediction on X test set
y_pred3 = model3.predict(X_test) #model prediction on X test set
y_pred4 = model4.predict(X_test) #model prediction on X test set


#Ensemble of 4 ANNs + Gradient Boost Decision Tree Emsemble Regressor
y_pred = ((0.1*(y_pred1 + y_pred2 + y_pred3 + y_pred4 + y_pred5) + 0.5*y_pred6))

#y_pred = ((y_pred5 + y_pred6)/2.0)
#y_test = np.reshape(y_test,(1567,1)) #no y_test
#print (y_pred.shape)
#print (y_test.shape)

#print(math.sqrt(mean_squared_error(y_pred, y)))
testing_labels = np.array(pd.read_csv('scoring.csv', delimiter = ',')['segment_id']) #testing data frame
testing_labels = np.reshape(testing_labels, (1567,1))
final = np.concatenate((testing_labels, y_pred), axis=1) #test is actual, pred is prediction
final = pd.DataFrame(final)
final.columns = ['segment_id', 'rate_of_penetration']
#export final prediction csv
export_csv = final.to_csv ('submission.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path



"""
#summarize history for rmse
plt.plot(history.history['rmse'])
plt.plot(history.history['val_rmse'])
plt.title('Model RMSE')
plt.ylabel('RMSE')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()


#cross val

from sklearn.model_selection import KFold #K fold cross validation 
#X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
#y = np.array([1, 2, 3, 4])
kf = KFold(n_splits=10)
kf.get_n_splits(X)

for train_index, test_index in kf.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
   
   #lsvc = linear_model.Lasso(alpha=1, max_iter = 1000).fit(X_train, y_train)
    #y_pred = lsvc.predict(X_test)
    #regressor = RandomForestRegressor(n_estimators=100, max_depth = 15, random_state=0)
    #regressor.fit(X_train, y_train)
    #y_pred = regressor.predict(X_test)
    #print (y_pred)
    #print (y_test)
    #print (np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    
    
    model = Sequential()
    model.add(Dense(2, input_dim=311, activation='linear'))
    model.add(Dense(5, kernel_initializer='normal',activation='linear'))
    model.add(Dense(5, kernel_initializer='normal',activation='linear'))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam', metrics=[rmse])
    history = model.fit(X_train, y_train, epochs=500, batch_size=100, verbose=2)

"""

segment_id               6838
wellbore_chev_no_id        21
area_id                     2
formation_id              239
bit_model_id               38
drillbit_size               7
min_depth                 990
max_depth                1542
rate_of_penetration      6838
surface_weight_on_bit    6837
surface_rpm              6210
dtype: int64
segment_id               1567
wellbore_chev_no_id         4
area_id                     2
formation_id              118
bit_model_id               11
drillbit_size               6
min_depth                 836
max_depth                 840
surface_weight_on_bit    1567
surface_rpm              1465
dtype: int64
(6838, 10)


'\n#summarize history for rmse\nplt.plot(history.history[\'rmse\'])\nplt.plot(history.history[\'val_rmse\'])\nplt.title(\'Model RMSE\')\nplt.ylabel(\'RMSE\')\nplt.xlabel(\'Epoch\')\nplt.legend([\'train\', \'validation\'], loc=\'upper left\')\nplt.show()\n\n# summarize history for loss\nplt.plot(history.history[\'loss\'])\nplt.plot(history.history[\'val_loss\'])\nplt.title(\'Model Loss\')\nplt.ylabel(\'Loss\')\nplt.xlabel(\'Epoch\')\nplt.legend([\'train\', \'validation\'], loc=\'upper left\')\nplt.show()\n\n\n#cross val\n\nfrom sklearn.model_selection import KFold #K fold cross validation \n#X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])\n#y = np.array([1, 2, 3, 4])\nkf = KFold(n_splits=10)\nkf.get_n_splits(X)\n\nfor train_index, test_index in kf.split(X):\n    #print("TRAIN:", train_index, "TEST:", test_index)\n    X_train, X_test = X[train_index], X[test_index]\n    y_train, y_test = y[train_index], y[test_index]\n   \n   #lsvc = linear_model.Lasso(alpha=1, max_iter = 1000).fit(X_trai